# Purity Reviewer Example

In [1]:
%load_ext autoreload
%autoreload 2
    

In [2]:
from PurityReviewer.Reviewers.MatchedPurityReviewer import MatchedPurityReviewer
from PurityReviewer.Reviewers.ManualPurityReviewer import ManualPurityReviewer
from PurityReviewer.AppComponents.utils import download_rdata
import pandas as pd
import numpy as np
import dalmatian
import os

# Set up simuated tumor data

If `SimulatedTumorData` submodule is not in the `example_notebooks` directory, clone the `SimulatedTumorData` repository in the `example_notebooks` directory
```
cd example_notebooks
git submodle add https://github.com/getzlab/SimulatedTumorData
```

In [3]:
# Run to install packages for loading the patient and sample data. Only need to run once in your environment.
%pip install -e SimulatedTumorData/.

Obtaining file:///Users/cchu/Desktop/Methods/PurityReviewers/example_notebooks/SimulatedTumorData
  Preparing metadata (setup.py) ... done
  Attempting uninstall: SimutatedTumorData
    Found existing installation: SimutatedTumorData 0.0.1
    Uninstalling SimutatedTumorData-0.0.1:
      Successfully uninstalled SimutatedTumorData-0.0.1
  Running setup.py develop for SimutatedTumorData
Note: you may need to restart the kernel to use updated packages.


## Load data

In [6]:
from SimulatedTumorData.src.generate_simulated_data import load_patients_and_samples

In [7]:
samples, participants = load_patients_and_samples(
    path_to_sim_data='SimulatedTumorData/sim_data'
)

/Users/cchu/Desktop/Methods/SimulatedTumorData/sim_data/patient1/phylogicNDT_results_1000
loading existing CNV pickle file SimulatedTumorData/sim_data/patient1/patient1.cnv_events.pkl
SimulatedTumorData/sim_data/patient1/sample_coverage/p1_t1.binned_coverage.tsv already exists.
SimulatedTumorData/sim_data/patient1/sample_coverage/p1_t2.binned_coverage.tsv already exists.
SimulatedTumorData/sim_data/patient1/sample_coverage/p1_t3.binned_coverage.tsv already exists.
patient variants path exists: SimulatedTumorData/sim_data/patient1/patient1.variants.tsv
Sample p1_t1 has variants_fn: SimulatedTumorData/sim_data/patient1/sample_muts/p1_t1.variants.tsv
Sample p1_t2 has variants_fn: SimulatedTumorData/sim_data/patient1/sample_muts/p1_t2.variants.tsv
Sample p1_t3 has variants_fn: SimulatedTumorData/sim_data/patient1/sample_muts/p1_t3.variants.tsv
Run SimulatedTumorData/sim_data/patient1/sample_mut_vcf/p1_t1.variants.vcf through (nexus-snp hg19 RefSeq).
Generated sif file: SimulatedTumorData/s

In [8]:
samples['ABSOLUTE_RData'].iloc[0]

'SimulatedTumorData/sim_data/patient1/sample_ABSOLUTE_results/p1_t1/ABSOLUTE_results/p1_t1.ABSOLUTE.RData'

In [9]:
samples

,maf_fn,wxs_purity,collection_date_dfd,cnv_seg_fn,participant_id,preservation_method,wxs_ploidy,ABSOLUTE_pp_calls_tab_fn,ABSOLUTE_pp_modes_data_fn,ABSOLUTE_pp_modes_plots_fn,ABSOLUTE_mode_res_rds_fn,ABSOLUTE_mode_tab_fn,ABSOLUTE_plot_fn,ABSOLUTE_SSNV_mode_res_rds_fn,ABSOLUTE_RData
sample_id,,,,,,,,,,,,,,,
p1_t1,/Users/cchu/Desktop/Methods/SimulatedTumorData...,0.70,50,SimulatedTumorData/sim_data/patient1/sample_cn...,patient1,NaN,1.83,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...
p1_t2,/Users/cchu/Desktop/Methods/SimulatedTumorData...,0.45,100,SimulatedTumorData/sim_data/patient1/sample_cn...,patient1,NaN,1.90,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...
p1_t3,/Users/cchu/Desktop/Methods/SimulatedTumorData...,0.90,120,SimulatedTumorData/sim_data/patient1/sample_cn...,patient1,NaN,1.75,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...,SimulatedTumorData/sim_data/patient1/sample_AB...
p2_t1,/Users/cchu/Desktop/Methods/SimulatedTumorData...,0.50,100,SimulatedTumorData/sim_data/patient2/sample_cn...,patient2,NaN,1.99,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...
p2_t2,/Users/cchu/Desktop/Methods/SimulatedTumorData...,0.85,150,SimulatedTumorData/sim_data/patient2/sample_cn...,patient2,NaN,1.99,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...
p2_t3,/Users/cchu/Desktop/Methods/SimulatedTumorData...,0.20,180,SimulatedTumorData/sim_data/patient2/sample_cn...,patient2,NaN,2.00,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...,SimulatedTumorData/sim_data/patient2/sample_AB...


In [10]:
participants

,maf_fn,cluster_ccfs_fn,build_tree_posterior_fn,tumor_molecular_subtype,tumor_morphology,tumor_primary_site,cancer_stage,vital_status,death_date_dfd,follow_up_date,age_at_diagnosis,gender,notes,treatments_fn
participant_id,,,,,,,,,,,,,,
patient1,SimulatedTumorData/sim_data/patient1/phylogicN...,SimulatedTumorData/sim_data/patient1/phylogicN...,SimulatedTumorData/sim_data/patient1/phylogicN...,Unknown,Unknown,NaN,NaN,NaN,NaN,NaN,32,NaN,NaN,SimulatedTumorData/sim_data/patient1/patient1_...
patient2,SimulatedTumorData/sim_data/patient2/phylogicN...,SimulatedTumorData/sim_data/patient2/phylogicN...,SimulatedTumorData/sim_data/patient2/phylogicN...,Unknown,Unknown,NaN,NaN,NaN,NaN,NaN,32,NaN,NaN,SimulatedTumorData/sim_data/patient2/patient2_...


# Reviewer

In [11]:
from PurityReviewer.AppComponents.utils import parse_absolute_soln, CSIZE_DEFAULT
from PurityReviewer.AppComponents.utils import parse_absolute_soln_simulatedTumorData

In [12]:
data_pkl_fn = 'simulated_data_purity_review.pkl'

In [13]:
matched_reviewer = MatchedPurityReviewer()
matched_reviewer.set_review_data(
    data_pkl_fn = data_pkl_fn, 
    description='Matched purity reviewer for simulated data', 
    df=samples, #pcyc_wm_pairs_df, # optional if directory above already exists. 
    index=samples.index, #pcyc_wm_pairs_df.index,
)
matched_reviewer.set_review_app(
    sample_info_cols=['ABSOLUTE_plot_fn', 'wxs_purity', 'wxs_ploidy'],
    acs_col='cnv_seg_fn', 
    maf_col='maf_fn',
    rdata_fn_col='ABSOLUTE_RData',
    mut_fig_hover_data=['Hugo_Symbol', 'Chromosome', 'Start_position'],
    csize=CSIZE_DEFAULT,
    custom_parse_absolute_soln=parse_absolute_soln_simulatedTumorData # <-- update with my_custom_parse_absolute_soln()
)

matched_reviewer.set_default_review_data_annotations_configuration()
matched_reviewer.set_default_autofill()

/Users/cchu/Desktop/Methods/AnnoMate/AnnoMate/ReviewDataInterface.py:38: UserWarning: Loading existing data pkl file
  warnings.warn(f"Loading existing data pkl file")


In [14]:
matched_reviewer.run(port=8099, mode='tab', collapsable=False, hide_history_df_cols=['source_data_fn'])

Setting auto_export_path to simulated_data_purity_review.auto_export
Using simulated_data_purity_review.auto_export for auto exporting.
Dash app running on http://0.0.0.0:8099/


<IPython.core.display.Javascript object>

In [19]:
fn = samples['ABSOLUTE_mode_tab_fn'].iloc[0]

In [25]:
samples.columns

Index(['maf_fn', 'wxs_purity', 'collection_date_dfd', 'cnv_seg_fn',
       'participant_id', 'preservation_method', 'wxs_ploidy',
       'ABSOLUTE_pp_calls_tab_fn', 'ABSOLUTE_pp_modes_data_fn',
       'ABSOLUTE_pp_modes_plots_fn', 'ABSOLUTE_mode_res_rds_fn',
       'ABSOLUTE_mode_tab_fn', 'ABSOLUTE_plot_fn',
       'ABSOLUTE_SSNV_mode_res_rds_fn', 'ABSOLUTE_RData'],
      dtype='object')

In [32]:
fn = samples['maf_fn'].iloc[0]
print(fn)
df = pd.read_csv(fn, sep='\t', encoding='iso-8859-1')

/Users/cchu/Desktop/Methods/SimulatedTumorData/sim_data/patient1/sample_muts_annotated/p1_t1.variants.annotated.tsv


In [34]:
import plotly.express as px

In [37]:
px.scatter(df, x='t_alt_count', y='t_ref_count', width=800, height=400)

In [39]:
from scipy.stats import beta

In [ ]:
x = np.linspace(beta.ppf(0.01, a, b),
                beta.ppf(0.99, a, b), 100)

In [ ]:
x, beta.pdf(x, a, b)

In [49]:
def get_beta_distr(a, b, x=None):
    
    x = np.linspace(beta.ppf(0.01, a, b),
                beta.ppf(0.99, a, b), 100) if x is None else x

    return pd.Series(index=x, data=beta.pdf(x, a, b))

In [53]:
step = 0.05
x = np.round(np.arange(0, 1 + step, step), 2)

In [54]:
df.loc[:, x] = df.apply(lambda r: get_beta_distr(a=r['t_alt_count'] + 1, b=r['t_ref_count'] + 1, x=x), axis=1)

In [55]:
df

,Hugo_Symbol,Chromosome,Start_position,Reference_Allele,Tumor_Seq_Allele2,Variant_Type,t_alt_count,t_ref_count,n_alt_count,n_ref_count,...,0.9,0.9500000000000001,1.0,0.15,0.3,0.35,0.6,0.7,0.85,0.95
0,FGD2,6,36993628,A,T,SNP,0.0,100.0,0,100.0,...,1.010000e-98,7.967495e-129,0.0,8.835150e-06,3.266821e-14,1.975409e-17,1.623007e-38,5.205313e-51,4.106268e-81,7.967495e-129
1,C19orf57,19,14000618,C,G,SNP,37.0,104.0,0,141.0,...,3.855295e-70,1.405202e-100,0.0,2.844655e-02,6.649169e+00,8.965851e-01,4.840834e-14,1.473407e-24,9.573616e-53,1.405202e-100
2,PCNXL2,1,233394701,A,T,SNP,28.0,50.0,0,78.0,...,5.049081e-29,2.037902e-43,0.0,2.431776e-04,3.969348e+00,7.311309e+00,7.510288e-04,3.185884e-08,6.497031e-21,2.037902e-43
3,RANBP6,9,6014284,A,G,SNP,43.0,77.0,0,120.0,...,9.943618e-45,6.728556e-67,0.0,1.265604e-06,3.579971e+00,9.002661e+00,6.084706e-06,1.103237e-12,3.084305e-32,6.728556e-67
4,L3MBTL1,20,42143973,C,G,SNP,48.0,116.0,0,164.0,...,8.193065e-75,1.321477e-108,0.0,2.370182e-04,1.104076e+01,3.334429e+00,1.995377e-13,1.048522e-24,1.407669e-55,1.321477e-108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,MBTPS2,23,21875555,G,C,SNP,0.0,151.0,0,151.0,...,1.520000e-149,5.324934e-195,0.0,3.342731e-09,6.189387e-22,8.545952e-27,1.238518e-58,1.687147e-77,5.910493e-123,5.324934e-195
82,USP43,17,9603498,T,A,SNP,0.0,120.0,0,120.0,...,1.210000e-118,9.103028e-155,0.0,4.102574e-07,3.122843e-17,4.289317e-21,2.137885e-46,2.174382e-61,1.635824e-97,9.103028e-155
83,ARHGAP21,10,24923955,C,T,SNP,0.0,120.0,0,120.0,...,1.210000e-118,9.103028e-155,0.0,4.102574e-07,3.122843e-17,4.289317e-21,2.137885e-46,2.174382e-61,1.635824e-97,9.103028e-155
84,TOP3A,17,18218028,G,C,SNP,7.0,113.0,0,120.0,...,3.442779e-101,4.840468e-135,0.0,1.300736e-01,4.933316e-09,3.348862e-12,2.172948e-34,4.870793e-48,1.825813e-81,4.840468e-135


In [ ]:
def plot_marginal_betas(df, a_col, b_col)


In [33]:
df

,Hugo_Symbol,Chromosome,Start_position,Reference_Allele,Tumor_Seq_Allele2,Variant_Type,t_alt_count,t_ref_count,n_alt_count,n_ref_count,...,CDNA Position,CDS Position,AA Position,AA Change,Detail,Splice Distance,Proteins,Variant_Classification,Protein_Change,UniProt_AApos
0,FGD2,6,36993628,A,T,SNP,0.0,100.0,0,100.0,...,1690,1519,507,N>Y,nonsyn,None,MKGASEEKLASVSNLVTVFENSRTPEAAPRGQRLEDVHHRPECRPP...,nonsyn,p.N507Y,507.0
1,C19orf57,19,14000618,C,G,SNP,37.0,104.0,0,141.0,...,None,None,None,None,NaN,2981,None,intronic,NaN,NaN
2,PCNXL2,1,233394701,A,T,SNP,28.0,50.0,0,78.0,...,1142,907,303,S>T,nonsyn,None,MVSQVLQLLRQGVWAALTGGWYHDPEQSKFTNSCHLYLWLFLLLLP...,nonsyn,p.S303T,303.0
3,RANBP6,9,6014284,A,G,SNP,43.0,77.0,0,120.0,...,242,None,None,None,NaN,None,None,3utr,NaN,NaN
4,L3MBTL1,20,42143973,C,G,SNP,48.0,116.0,0,164.0,...,534,425,142,S>C,nonsyn,None,MRRREGHGTDSEMGQGPVRESQSSDPPALQFRISEYKPLNMAGVEQ...,nonsyn,p.S142C,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,MBTPS2,23,21875555,G,C,SNP,0.0,151.0,0,151.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,USP43,17,9603498,T,A,SNP,0.0,120.0,0,120.0,...,1563,1467,489,H>Q,nonsyn,None,MDLGPGDAAGGGPLAPRPRRRRSLRRLFSRFLLALGSRSRPGDSPP...,nonsyn,p.H489Q,489.0
83,ARHGAP21,10,24923955,C,T,SNP,0.0,120.0,0,120.0,...,833,346,116,A>T,nonsyn,None,MMATRRTGLSEGDGDKLKACEVSKNKDGKEQSETVSLSEDETFSWP...,nonsyn,p.A116T,116.0
84,TOP3A,17,18218028,G,C,SNP,7.0,113.0,0,120.0,...,None,None,None,None,NaN,None,None,5upstream,NaN,NaN


# Purity Review from Terra

In [7]:
wm = dalmatian.WorkspaceManager('broad-getzlab-ibm-taml-t/TCGA_LUAD_Tonly2_Validation')
pairs_df = wm.get_pairs()
luad_pairs_df = pairs_df[pairs_df['absolute_rdata_WGS'].notna()].head(5).copy()

In [8]:
[c for c in pairs_df.columns.tolist() if 'mutation_validator_validated_maf' in c]

['mutation_validator_validated_maf_make_forcecall_intervals',
 'mutation_validator_validated_maf_WGS',
 'mutation_validator_validated_maf_make_forcecall_intervals_forcecalled_snps_and_indels_maf']

In [9]:
import os
os.environ["GCLOUD_PROJECT"] = "broad-getzlab-ibm-taml"

In [10]:
# download rdata locally
rdata_dir = f'/Users/cchu/Desktop/Methods/PurityReviewers/example_notebooks/data/luad_local_rdata_2023-09-11'
downloaded_rdata_s = download_rdata(luad_pairs_df['absolute_rdata_WGS'], rdata_dir=rdata_dir)
downloaded_rdata_s.name = 'local_absolute_rdata'
luad_pairs_df = pd.concat([luad_pairs_df, downloaded_rdata_s], axis=1)

In [12]:
matched_reviewer = MatchedPurityReviewer()
matched_reviewer.set_review_data(
    data_pkl_fn = 'luad.review_data.pkl', 
    description='Matched purity reviewer', 
    df=luad_pairs_df, # optional if directory above already exists. 
    index=luad_pairs_df.index,
)
matched_reviewer.set_review_app(
    sample_info_cols=['participant', 'alleliccapseg_plot_WGS'],
    acs_col='alleliccapseg_tsv_WGS', 
    maf_col='mutation_validator_validated_maf_WGS',
    rdata_fn_col='local_absolute_rdata',
    mut_fig_hover_data=['Hugo_Symbol', 'Chromosome', 'Start_position'],
    csize=CSIZE_DEFAULT,
    custom_parse_absolute_soln=parse_absolute_soln # <-- update with my_custom_parse_absolute_soln()
)

matched_reviewer.set_default_review_data_annotations_configuration()
matched_reviewer.set_default_autofill()

In [14]:
matched_reviewer.run(port=8099, review_data_table_df=luad_pairs_df[['Purity', 'participant']], mode='tab')

Setting auto_export_path to luad.review_data.auto_export
Making directory luad.review_data.auto_export for auto exporting.
Using luad.review_data.auto_export for auto exporting.
Dash app running on http://0.0.0.0:8099/


/Users/cchu/opt/anaconda3/envs/purity_reviewer_annomate_env/lib/python3.9/site-packages/AnnoMate/ReviewDataApp.py:685: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  review_data_table_data = new_review_data_table_df.reset_index().to_dict('records')


<IPython.core.display.Javascript object>

/Users/cchu/opt/anaconda3/envs/purity_reviewer_annomate_env/lib/python3.9/site-packages/AnnoMate/AppComponents/utils.py:102: DtypeWarning:

Columns (38,56,72,73,74,84,88,91,92,95,100,102,104,105,106,107,109,110,111,113,114,121,122,147) have mixed types. Specify dtype option on import or set low_memory=False.



---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File ~/opt/anaconda3/envs/purity_reviewer_annomate_env/lib/python3.9/site-packages/AnnoMate/ReviewDataApp.py:325, in ReviewDataApp.run.<locals>.update_components(
    output_dict={'more_component_outputs': {'Absolute Solutions': [<dash._callback.NoUpdate object>, <dash._callback.NoUpdate object>, <dash._callback.NoUpdate object>, <dash._callback.NoUpdate object>, <dash._callback.NoUpdate object>, <dash._callback.NoUpdate object>, <dash._callback.NoUpdate object>], 'Annotated Data Information table': [<dash._callback.NoUpdate object>], 'Manual Purity': [<dash._callback.NoUpdate object>, <dash._callback.NoUpdate object>, <dash._callback.NoUpdate object>, <dash._callback.NoUpdate object>, <dash._callback.NoUpdate object>, <dash._callback.NoUpdate object>]}, 'review_data_page_current': 0, 'review_data_selected_value': Int64Index([0], dtype=